# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 49 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.01775


extracting tarball to tmp_2301.01775...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.01776


extracting tarball to tmp_2301.01776...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.02157


extracting tarball to tmp_2301.02157...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.02179


extracting tarball to tmp_2301.02179...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01895-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01895) | **ATOMS: ALMA Three-millimeter Observations of Massive Star-forming  regions -- XV. Steady Accretion from Global Collapse to Core Feeding in  Massive Hub-filament System SDC335**  |
|| Feng-Wei Xu, et al. -- incl., <mark>Tie Liu</mark>, <mark>Qizhou Zhang</mark>, <mark>Hong-Li Liu</mark>, <mark>Sheng-Li Qin</mark>, <mark>Guang-Xing Li</mark>, <mark>Shanghuo Li</mark>, <mark>Enrique Vázquez-Semadeni</mark>, <mark>Yong Zhang</mark>, <mark>Xunchuan Liu</mark>, <mark>Siju Zhang</mark>, <mark>Meizhu Liu</mark>, <mark>Chao Zhang</mark>, <mark>Chang Won Lee</mark>, <mark>Jinjin Xie</mark>, <mark>Wenyu Jiao</mark>, <mark>Rong Liu</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *29 pages, 7 figures, Accepted for publication in MNRAS*|
|**Abstract**| We present ALMA Band-3/7 observations towards "the Heart" of a massive hub-filament system (HFS) SDC335, to investigate its fragmentation and accretion. At a resolution of $\sim0.03$ pc, 3 mm continuum emission resolves two massive dense cores MM1 and MM2, with $383(^{+234}_{-120})$ $M_\odot$ (10-24% mass of "the Heart") and $74(^{+47}_{-24})$ $M_\odot$, respectively. With a resolution down to 0.01 pc, 0.87 mm continuum emission shows MM1 further fragments into six condensations and multi-transition lines of H$_2$CS provide temperature estimation. The relation between separation and mass of condensations at a scale of 0.01 pc favors turbulent Jeans fragmentation where the turbulence seems to be scale-free rather than scale-dependent. We use the H$^{13}$CO$^+$ (1-0) emission line to resolve the complex gas motion inside "the Heart" in position-position-velocity space. We identify four major gas streams connected to large-scale filaments, inheriting the anti-clockwise spiral pattern. Along these streams, gas feeds the central massive core MM1. Assuming an inclination angle of $45(\pm15)^{\circ}$ and a H$^{13}$CO$^+$ abundance of $5(\pm3)\times10^{-11}$, the total mass infall rate is estimated to be $2.40(\pm0.78)\times10^{-3}$ $M_\odot$ yr$^{-1}$, numerically consistent with the accretion rates derived from the clump-scale spherical infall model and the core-scale outflows. The consistency suggests a continuous, near steady-state, and efficient accretion from global collapse, therefore ensuring core feeding. Our comprehensive study of SDC335 showcases the detailed gas kinematics in a prototypical massive infalling clump and calls for further systematic and statistical analyses in a large sample. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01937-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01937) | **ATOMS: ALMA Three-millimeter Observations of Massive Star-forming  regions -XIII. Ongoing triggered star formation within clump-fed scenario  found in the massive ($\sim1500$ $\rm M_\odot$) clump**  |
|| <mark>S. Zhang</mark>, et al. -- incl., <mark>T. Liu</mark>, <mark>H. Liu</mark>, <mark>S. Li</mark>, <mark>Q. Zhang</mark>, <mark>C. W. Lee</mark>, <mark>E. Vázquez-Semadeni</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *34 pages with 20 figures, Accepted by MNRAS on 2022 December 28*|
|**Abstract**| Whether ionization feedback triggers the formation of massive stars is highly debated. Using ALMA 3 mm observations with a spatial resolution of $\sim 0.05$ pc and a mass sensitivity of 1.1 $\rm M_\odot$ beam$^{-1}$ at 20 K, we investigate the star formation and gas flow structures within the ionizing feedback-driven structure, a clump-scale massive ($\gtrsim 1500$ $\rm M_\odot$) bright-rimmed cloud (BRC) associated with IRAS 18290-0924. This BRC is bound only if external compression from ionized gas is considered. A small-scale ($\lesssim1$ pc) age sequence along the direction of ionizing radiation is revealed for the embedded cores and protostars, which suggests triggered star formation via radiation-driven implosion (RDI). Furthermore, filamentary gas structures converge towards the cores located in the BRC's center, indicating that these filaments are fueling mass towards cores. The local core-scale mass infall rate derived from H$^{13}$CO$^+$ $J=1-0$ blue profile is of the same order of magnitude as the filamentary mass inflow rate, approximately 1 $\rm M_\odot$ kyr$^{-1}$. A photodissociation region (PDR) covering the irradiated clump surface is detected in several molecules, such as CCH, HCO$^+$, and CS whereas the spatial distribution stratification of these molecules is indistinct. CCH spectra of the PDR possibly indicate a photoevaporation flow leaving the clump surface with a projected velocity of $\sim2$ km s$^{-1}$. Our new observations show that RDI accompanied by a clump-fed process is operating in this massive BRC. Whether this combined process works in other massive BRCs is worth exploring with dedicated surveys. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02081-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02081) | **The VLT/SPHERE view of the ATOMIUM cool evolved star sample. I.  Overview: Sample characterization through polarization analysis**  |
|| M. Montargès, et al. -- incl., <mark>F. Herpin</mark>, <mark>H. S. P. Müller</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics. 22 pages, 15 figures, 5 tables*|
|**Abstract**| Aims. Through the ATOMIUM project, based on an ALMA large program, we aim to present a consistent view of a sample of 17 nearby cool evolved stars (Aymptotic Giant Branch and red supergiant stars). Methods. Here we present VLT/SPHERE-ZIMPOL polarimetric maps obtained in the visible of 14 out of the 17 ATOMIUM sources. They were obtained contemporaneously with the ALMA high spatial resolution data. To help interpret the polarized signal, we produced synthetic maps of light scattering by dust, through 3D radiative transfer simulations with the RADMC3D code. Results. The degree of linear polarization (DoLP) observed by ZIMPOL spreads across several optical filters. We infer that it primarily probes dust located just outside of the point spread function, and in or near the plane of the sky, with a total optical depth close to unity in the line of sight, representing only a fraction of the total circumstellar dust. The maximum DoLP ranges from 0.03-0.38 depending on the source, fractions that can be reproduced by our 3D pilot models for grains composed of common dust species. The spatial structure of the DoLP shows a diverse set of shapes. Only for three sources do we note a correlation between the ALMA CO and SiO lines, which trace the gas density, and the DoLP, which traces the dust. Conclusion. The clumpiness of the DoLP and the lack of a consistent correlation between the gas and the dust location show that, in the inner circumstellar environment (CSE), dust formation occurs at very specific sites. This has potential consequences for the derived mass-loss rates and dust-to-gas ratio in the inner region of the CSE. Except for $\pi^1$~Gru and perhaps GY Aql, we do not detect interactions between the circumstellar wind and the hypothesized companions that shape the wind at larger scales. This suggests that the orbits of any other companions are tilted out of the plane of the sky. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02070-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02070) | **Fragmentation of the High-mass "Starless'' Core G10.21-0.31: a Coherent  Evolutionary Picture for Star Formation**  |
|| <mark>Wenyu Jiao</mark>, et al. -- incl., <mark>Siju Zhang</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *30 pages, 13 figures; accepted for publication in ApJ*|
|**Abstract**| G10.21-0.31 is a 70 $\mu$m-dark high-mass starless core ($M>300$ $\mathrm{M_{\odot}}$ within $r<0.15$ pc) identified in $Spitzer$, $Herschel$, and APEX continuum surveys, and is believed to harbor the initial stages of high-mass star formation. We present ALMA and SMA observations to resolve the internal structure of this promising high-mass starless core. Sensitive high-resolution ALMA 1.3 mm dust continuum emission reveals three cores of mass ranging 11-18 $\mathrm{M_{\odot}}$, characterized by a turbulent fragmentation. Core 1, 2, and 3 represent a coherent evolution at three different evolutionary stages, characterized by outflows (CO, SiO), gas temperature ($\mathrm{H_2CO}$), and deuteration ($\mathrm{N_2D^+/N_2H^+}$). We confirm the potential to form high-mass stars in G10.21 and explore the evolution path of high-mass star formation. Yet, no high-mass prestellar core is present in G10.21. This suggests a dynamical star formation where cores grow in mass over time. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02157-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02157) | **Asteroids' reflectance from Gaia DR3: Artificial reddening at near-UV  wavelengths**  |
|| Fernando Tinaut-Ruano, et al. -- incl., <mark>Javier Licandro</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *5 pages plus appendix. Accepted for publication in A&A*|
|**Abstract**| Aims. We aim to assess the suitability in the near-ultraviolet (NUV) region of the solar analogues selected by the team responsible for the asteroid reflectance included in Gaia Data Release 3 (DR3) and to suggest a correction (in the form of multiplicative factors) to be applied to the Gaia DR3 asteroid reflectance spectra to account for the differences with respect to the solar analogue Hyades 64. Results. We find that the solar analogues selected for Gaia DR3 to compute the reflectance spectra of the asteroids of this data release have a systematically redder spectral slope at wavelengths shorter than 0.55 {\mu}m than Hyades 64. We find that no correction is needed in the red photometer (RP, between 0.7 and 1 {\mu}m), but a correction should be applied at wavelengths below 0.55 {\mu}m, that is in the blue photometer (BP). After applying the correction, we find a better agreement between Gaia DR3 spectra, ECAS, HST, and our set of ground-based observations with the TNG. Conclusions. Correcting the near-UV part of the asteroid reflectance spectra is very important for proper comparisons with laboratory spectra (minerals, meteorite samples, etc.) or to analyse quantitatively the UV absorption (which is particularly important to study hydration in primitive asteroids). The spectral behaviour at wavelengths below 0.5 {\mu}m of the selected solar analogues should be fully studied and taken into account for Gaia DR4 |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02179-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02179) | **The GLASS-JWST Early Release Science Program. II. Stage I release of  NIRCam imaging and catalogs in the Abell 2744 region**  |
|| Diego Paris, et al. -- incl., <mark>Lilan Yang</mark>, <mark>Nicha Leethochawalit</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**| We present images and a multi-wavelength photometric catalog based on all of the JWST NIRCam observations obtained to date in the region of the Abell 2744 galaxy cluster. These data come from three different programs, namely the GLASS-JWST Early Release Science Program, UNCOVER, and Director's Discretionary Time program 2756. The observed area in the NIRCam wide-band filters - covering the central and extended regions of the cluster, as well as new parallel fields - is 46.5 arcmin$^2$ in total. All images in eight bands (F090W, F115W, F150W, F200W, F277W, F356W, F410M, F444W) have been reduced adopting the latest calibration and references available to date. Data reduction has been performed using an augmented version of the official JWST pipeline, with improvements aimed at removing or mitigating defects in the raw images and improve the background subtraction and photometric accuracy. We obtain a F444W-detected multi-band catalog including all NIRCam data and available HST data, adopting forced aperture photometry on PSF-matched images. The catalog is intended to enable early scientific investigations, and is optimized for the study of faint galaxies; it contains 24389 sources, with a 5$\sigma$ limiting magnitude in the F444W band ranging from 28.5 to 30.5 AB, as a result of the varying exposure times of the surveys that observed the field. We publicly release the reduced NIRCam images, associated multi-wavelength catalog, and code adopted for $1/f$ noise removal with tha aim of aiding users to familiarize themselves with JWST NIRCam data and identify targets for follow-up observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01944-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01944) | **Study of variability in long-term multiwavelength optical lightcurves of  blazar AO 0235+164**  |
|| Abhradeep Roy, et al. -- incl., <mark>Mai Liao</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| *27 pages, 10 figures. Accepted for publication in The Astrophysical Journal Supplement Series (ApJS)*|
|**Abstract**| We present a long-term and intraday variability study on optical multiwaveband ($U\!BV\!RI$) data from the blazar AO 0235+164 collected by various telescopes for $\sim$44 years (1975--2019). The blazar was found to be significantly variable over the years in all wavebands with a variation of about six magnitudes between its low and active states. The variations in the different wavebands are highly correlated without any time lag. We did not observe any significant trend in color variation with time, but we observed a bluer-when-brighter trend between the $B-I$ color index and the $R$-magnitude. Optical $BV\!R$-band spectral energy distributions always show a convex shape. Significant intraday variability was frequently seen in the quasi-simultaneous observations of AO\,0235+164 made on 22 nights in $R$ and $V$-bands by the CASLEO and CAHA telescopes during 1999--2019. We also estimated the central supermassive black-hole mass of $7.9\times10^7 M_{\odot}$ by analyzing the broad Mg II emission line in AO 0235+164's spectrum. We briefly explore the probable physical scenarios responsible for the observed variability. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.01876-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.01876) | **ATOMS: ALMA Three-millimeter Observations of Massive Star-forming  regions -XIV. Properties of resolved UC Hii regions**  |
|| <mark>C. Zhang</mark>, et al. -- incl., <mark>Tie Liu</mark>, <mark>H. -L. Liu</mark>, <mark>Y. Zhang</mark>, <mark>Shanghuo Li</mark>, <mark>Chang Won Lee</mark>, <mark>D. Li</mark> |
|*Appeared on*| *2023-01-06*|
|*Comments*| **|
|**Abstract**| Hydrogen recombination lines (RRLs) are one of the major diagnostics of the physical properties of H{\sc ii} regions. We use RRL H40$\alpha$, He40$\alpha$ and 3 mm continuum emission to investigate the properties of a large sample of resolved UC H{\sc ii} regions identified in the ATOMS survey. In total, we identify 94 UC H{\sc ii} regions from H40$\alpha$ emission. The basic parameters for these UC H{\sc ii} regions such as electron density, emission measure, electron temperature, ionic abundance ratio (n$_{\rm He^+}$/n$_{\rm H^+}$), and line width are derived. The median electron density and the median n$_{\rm He^+}$/n$_{\rm H^+}$ ratio of these UC H{\sc ii} regions derived from RRLs are $\sim$9000 cm$^{-3}$ and 0.11, respectively. Within UC H{\sc ii} regions, the n$_{\rm He^+}$/n$_{\rm H^+}$ ratios derived from the intensity ratio of the He40$\alpha$ and H40$\alpha$ lines seems to be higher in the boundary region than in the center. The H40$\alpha$ line width is mainly broadened by thermal motion and microturbulence. The electron temperature of these UC H{\sc ii} regions has a median value of $\sim$6700 K, and its dependence on galactocentric distance is weak. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.01895.md
    + _build/html/tmp_2301.01895/./fig/PyV.png
    + _build/html/tmp_2301.01895/./fig/PxV.png
    + _build/html/tmp_2301.01895/./fig/zoomin.png
exported in  _build/html/2301.01937.md
    + _build/html/tmp_2301.01937/./BRC-Statistics.png
    + _build/html/tmp_2301.01937/./PV-View.png
    + _build/html/tmp_2301.01937/./Large-Env-Improved.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\casa}{\textsc{\large casa}}$
$\newcommand{\getsf}{\textsc{\large getsf}}$
$\newcommand{\getsources}{\textsc{\large getsources}}$
$\newcommand{\scousepy}{\textsc{\large scousepy}}$
$\newcommand{\astrodendro}{\textsc{\large astrodendro}}$
$\newcommand{\acorns}{\textsc{\large acorns}}$
$\newcommand{\tablenotemarknew}[1]{^{#1}}$
$\newcommand{\majchange}[1]{{\color{red}#1}}$
$\newcommand{\minchange}[1]{\textbf{#1}}$
$\newcommand{\massrate}{M_{\odot} yr^{-1}}$
$\newcommand{\hi}{H\textsc{i}~}$
$\newcommand{\hii}{H\textsc{ii}~}$
$\newcommand{\msun}{ M_\odot}$
$\newcommand{\lsun}{ L_\odot}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\jybeam}{Jy beam^{-1}}$
$\newcommand{\mjybeam}{mJy beam^{-1}}$
$\newcommand{\hmole}{H_2}$
$\newcommand{\degree}{^{\circ}}$
$\newcommand{\parcsec}{\mbox{.\!\!\arcsec}}$
$\newcommand{\ssstyle}{\scriptscriptstyle}$
$\newcommand{\htcop}{H^{13}CO^+}$
$\newcommand{\hcop}{HCO^+}$
$\newcommand{\hcn}{HCN}$
$\newcommand{\htcn}{H^{13}CN}$
$\newcommand{\sio}{SiO}$
$\newcommand{\cch}{CCH}$
$\newcommand{\chtoh}{CH_3OH}$
$\newcommand{\hctn}{HC_3N}$
$\newcommand{\cs}{CS}$
$\newcommand{\so}{SO}$
$\newcommand{\htcs}{H_2CS}$
$\newcommand{\nthp}{N_2H^+}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\casa$}{$\textsc${\large casa}}$
$\newcommand{$\getsf$}{$\textsc${\large getsf}}$
$\newcommand{$\getsources$}{$\textsc${\large getsources}}$
$\newcommand{$\scousepy$}{$\textsc${\large scousepy}}$
$\newcommand{$\astrodendro$}{$\textsc${\large astrodendro}}$
$\newcommand{$\acorns$}{$\textsc${\large acorns}}$
$\newcommand{$\tablenotemarknew$}[1]{^{#1}}$
$\newcommand{$\majchange$}[1]{{\color{red}#1}}$
$\newcommand{$\minchange$}[1]{\textbf{#1}}$
$\newcommand{$\massrate$}{M_{\odot} yr^{-1}}$
$\newcommand{$\hi$}{H$\textsc${i}~}$
$\newcommand{$\hi$i}{H$\textsc${ii}~}$
$\newcommand{$\msun$}{ M_\odot}$
$\newcommand{$\lsun$}{ L_\odot}$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand{$\jybeam$}{Jy beam^{-1}}$
$\newcommand{$\mjybeam$}{mJy beam^{-1}}$
$\newcommand{$\hmole$}{H_2}$
$\newcommand{$\degree$}{^{\circ}}$
$\newcommand{$\parcsec$}{\mbox{.\!\!$\arcsec$}}$
$\newcommand{$\ssstyle$}{\scriptscriptstyle}$
$\newcommand{$\htcop$}{H^{13}CO^+}$
$\newcommand{$\hcop$}{HCO^+}$
$\newcommand{$\hcn$}{HCN}$
$\newcommand{$\htcn$}{H^{13}CN}$
$\newcommand{$\sio$}{SiO}$
$\newcommand{$\cch$}{CCH}$
$\newcommand{$\chtoh$}{CH_3OH}$
$\newcommand{$\hctn$}{HC_3N}$
$\newcommand{$\cs$}{CS}$
$\newcommand{$\so$}{SO}$
$\newcommand{$\htcs$}{H_2CS}$
$\newcommand{$\nthp$}{N_2H^+}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# ATOMS: ALMA Three-millimeter Observations of Massive Star-forming regions -- XV. Steady Accretion from Global Collapse to Core Feeding in Massive Hub-filament System SDC335

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.01895-b31b1b.svg)](https://arxiv.org/abs/2301.01895)<mark>Appeared on: 2023-01-06</mark> - _29 pages, 7 figures, Accepted for publication in MNRAS_

</div>
<div id="authors">

Feng-Wei Xu, et al. -- incl., <mark><mark>Tie Liu</mark></mark>, <mark><mark>Qizhou Zhang</mark></mark>, <mark><mark>Hong-Li Liu</mark></mark>, <mark><mark>Sheng-Li Qin</mark></mark>, <mark><mark>Guang-Xing Li</mark></mark>, <mark><mark>Shanghuo Li</mark></mark>, <mark><mark>Enrique Vázquez-Semadeni</mark></mark>, <mark><mark>Yong Zhang</mark></mark>, <mark><mark>Xunchuan Liu</mark></mark>, <mark><mark>Siju Zhang</mark></mark>, <mark><mark>Meizhu Liu</mark></mark>, <mark><mark>Chao Zhang</mark></mark>, <mark><mark>Chang Won Lee</mark></mark>, <mark><mark>Jinjin Xie</mark></mark>, <mark><mark>Wenyu Jiao</mark></mark>, <mark><mark>Rong Liu</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present ALMA Band-3/7 observations towards "the Heart" of a massive hub-filament system (HFS) SDC335, to investigate its fragmentation and accretion. At a resolution of$\sim0.03$pc, 3 mm continuum emission resolves two massive dense cores MM1 and MM2, with$383(^{$\ssstyle$+234}_{$\ssstyle$-120})$\msun(10--24\%mass of "the Heart") and$74(^{$\ssstyle$+47}_{$\ssstyle$-24})$\msun, respectively. With a resolution down to$0.01$pc, 0.87 mm continuum emission shows MM1 further fragments into six condensations and multi-transition lines of$\htcs$provide temperature estimation. The relation between separation and mass of condensations at a scale of 0.01 pc favors turbulent Jeans fragmentation where the turbulence seems to be scale-free rather than scale-dependent. We use the$\htcop$$J=1-0$emission line to resolve the complex gas motion inside "the Heart" in position-position-velocity space. We identify four major gas streams connected to large-scale filaments, inheriting the anti-clockwise spiral pattern. Along these streams, gas feed the central massive core MM1. Assuming an inclination angle of$45(\pm15)^{\circ}$and a$\htcop$abundance of$5(\pm3)\times10^{-11}$, the total mass infall rate is estimated to be$2.40(\pm0.78)\times10^{-3}$\massrate, numerically consistent with the accretion rates derived from the clump-scale spherical infall model and the core-scale outflows. The consistency suggests a continuous, near steady-state, and efficient accretion from global collapse, therefore ensuring core feeding. Our comprehensive study of SDC335 showcases the detailed gas kinematics in a prototypical massive infalling clump, and calls for further systematic and statistical studies in a large sample.

</div>

<div id="div_fig1">

<img src="tmp_2301.01895/./fig/PyV.png" alt="Fig13" width="100%"/>

**Figure 13. -** The "$\Delta$ Dec vs. Velocity" plot of four major streams are shown with scattering points. Each point represents one Gaussian component of the decomposed $\htcop$$J=1-0$ emission, with its color normalized by the $\Delta$ RA. The location of massive dense core MM1 is marked by the cyan color. Two major streams A and B are marked by orange contours. The three contour levels represent the $1\sigma$(solid line), $2\sigma$(dashed line), and $3\sigma$(dotted line) probability distribution of scatter-points. Similarly, minor streams C1 and D1 are marked by the orange contours and minor streams C2 and D2 are marked by the pink colors. In each panel, "F1--F6" marks the end of the large-scale filaments which are assumed to be responsible for the streams. The RA, DEC location as well as velocity of filament ends are measured from $\nthp$ data from \citet{2013A&A...555A.112P}. In the upper two panels, the stream A and B are respectively fitted by free-fall models (a central mass of 383 $\msun$) with inclination angles of $\theta=30^{\circ}$(solid black line; best fitting), $\theta=40^{\circ}$(dash-dotted black line), and $\theta=20^{\circ}$(dash-dotted gray line). The central circle marks how we visualize the 3D data cube: blue axis is the collapsing axis, where the data points are projected on the plane spanned by two black axes. \label{fig:PyV} (*fig:PyV*)

</div>
<div id="div_fig2">

<img src="tmp_2301.01895/./fig/PxV.png" alt="Fig14" width="100%"/>

**Figure 14. -** The "$\Delta$ RA vs. Velocity" plot of four major streams are shown with scattering points. Each point represents one Gaussian component of the decomposed $\htcop$$J=1-0$ emission, with its color normalized by the centroid velocity on a linear span from -50 to -42 $\kms$. The location of massive dense core MM1 is marked by the purple color. Two major streams A and B are marked by orange contours. The three contour levels represent the $1\sigma$(solid line), $2\sigma$(dashed line), and $3\sigma$(dotted line) probability distribution of scatter-points. Similarly, minor streams C1 and D1 are marked by the orange contours and minor streams C2 and D2 are marked by the pink colors. In each panel, "F1--F6" marks the end of the large-scale filaments which are assumed to be responsible for the streams. The RA, DEC location as well as velocity of filament ends are measured from $\nthp$ data from \citet{2013A&A...555A.112P}. The central circle marks how we visualize the 3D data cube: blue axis is the collapsing axis, where the data points are projected on the plane spanned by two black axes. \label{fig:PxV} (*fig:PxV*)

</div>
<div id="div_fig3">

<img src="tmp_2301.01895/./fig/zoomin.png" alt="Fig9" width="100%"/>

**Figure 9. -** Successive zoom-ins from (a) SDC335, to (b) "the Heart", and to (c) the massive dense core SDC335-MM1.
(a) The background color map is the composite (Red/Green/Blue: Spitzer 8/4.5/3.6 $\mu$m) images of SDC335, with Red/Green/Blue in logarithmic stretch. The ATLASGAL 870 $\mu$m continuum emission is overlaid as a single contour level of $5\sigma=0.75$\jybeam with the APEX beam of 21$\arcsec$. Six converging filaments F1--F6, identified from the Spitzer extinction and dense gas N$_2$H$^+$\citep{2013A&A...555A.112P}, are marked with yellow dashed lines. Two white circles show the ALMA Band-3 primary beam response of 12m+ACA combined data: the dashed one is 32$\arcsec$ for 20\% and the solid one is 46.5$\arcsec$ for 50\%. (b) The zoom-in version of the left panel towards "the Heart", with the same background color map. Two white circles show the ALMA primary beam responses respectively at 50\% for Band-3 (outer) and Band-7 (inner). The white contours of 3 mm continuum emission follows the power-law levels of [1.0, 3.3, 7.6, 14.2, 23.4, 35.1, 49.6, 67.0]$\mjybeam$. (c) The zoom-in version of the middle panel, towards SDC335-MM1. The white contours and the white circle are the same as those in the middle panel. The background color map shows the ALMA 0.87 mm continuum emission, in a square-root stretch to highlight the weak emission. The beams for ATLASGAL 870 $\mu$m, ALMA Band-3, or ALMA Band-7 are shown on the left bottom and the scale bars are shown on the right bottom. \label{fig:zoomin} (*fig:zoomin*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\hi}{H~\textsc{i}}$
$\newcommand{\hii}{H~\textsc{ii}}$
$\newcommand{\msun}{\rm M_\odot}$
$\newcommand{\lsun}{\rm L_\odot}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\jybeam}{Jy beam^{-1}}$
$\newcommand{\mjybeam}{mJy beam^{-1}}$
$\newcommand{\hmole}{H_2}$
$\newcommand{\cm}{cm^{-2}}$
$\newcommand{\pressure}{K~cm^{-3}}$
$\newcommand{\hcop}{HCO^+}$
$\newcommand{\hcopone}{HCO^+~J=1-0}$
$\newcommand{\htcop}{H^{13}CO^+}$
$\newcommand{\htcopone}{H^{13}CO^+~J=1-0}$
$\newcommand{\hcn}{HCN}$
$\newcommand{\hcnone}{HCN~J=1-0}$
$\newcommand{\htcn}{H^{13}CN}$
$\newcommand{\htcnone}{H^{13}CN~J=1-0}$
$\newcommand{\cs}{CS}$
$\newcommand{\cstwo}{CS~J=2-1}$
$\newcommand{\sio}{SiO}$
$\newcommand{\siotwo}{SiO~J=2-1}$
$\newcommand{\siofive}{SiO~J=5-4}$
$\newcommand{\hctn}{HC_3N}$
$\newcommand{\hctnele}{HC_3N~J=11-10}$
$\newcommand{\hfourtyalpha}{H40\alpha}$
$\newcommand{\chtoh}{CH_3OH}$
$\newcommand{\chtohline}{CH_3OH 2(1,1)-1(1,0)A}$
$\newcommand{\cch}{CCH}$
$\newcommand{\cchline}{CCH~N_{J,F}=1_{3/2,2}-0_{1/2,1}}$
$\newcommand{\cchhyperfine}{CCH~N_{J,F}=1_{3/2,1}-0_{1/2,0}}$
$\newcommand{\so}{SO}$
$\newcommand{\soline}{SO v =0, 3(2)-2(1)}$
$\newcommand{\nht}{NH_3}$
$\newcommand{\mclump}{M_{\rm clp}}$
$\newcommand{\rclump}{r_{\rm clp}}$
$\newcommand{\RGC}{R_{\rm GC}}$
$\newcommand{\mvir}{M_{\rm vir}}$
$\newcommand{\tkin}{T_{\rm kin}}$
$\newcommand{\tex}{T_{\rm ex}}$
$\newcommand{\tbg}{T_{\rm bg}}$
$\newcommand{\tr}{T_{\rm r}}$
$\newcommand{\te}{T_{\rm e}}$
$\newcommand{\hcopabundance}{\chi(\hcop)}$
$\newcommand{\hncabundance}{\chi(\hnc)}$
$\newcommand{\nthpabundance}{\chi(\nthp)}$
$\newcommand{\nthphcopabundance}{\chi(\hcop)/\chi(\nthp)}$
$\newcommand{\nthphncabundance}{\chi(\hnc)/\chi(\nthp)}$
$\newcommand{\tcoab}{\chi(^{13}CO)}$
$\newcommand{\ceoab}{\chi(C^{18}O)}$
$\newcommand{\co}{CO}$
$\newcommand{\tco}{^{13}CO}$
$\newcommand{\ceo}{C^{18}O}$
$\newcommand{\coone}{^{12}CO J=1-0}$
$\newcommand{\cotwo}{^{12}CO J=2-1}$
$\newcommand{\cothr}{^{12}CO J=3-2}$
$\newcommand{\tcoone}{^{13}CO J=1-0}$
$\newcommand{\tcotwo}{^{13}CO J=2-1}$
$\newcommand{\tcothr}{^{13}CO J=3-2}$
$\newcommand{\ceoone}{C^{18}O~J=1-0}$
$\newcommand{\ceotwo}{C^{18}O J=2-1}$
$\newcommand{\ceothr}{C^{18}O J=3-2}$
$\newcommand{\degree}{^{\circ}}$
$\newcommand{\nhtcd}{N_{\rm H_{2}}}$
$\newcommand{\dustt}{T_{\rm dust}}$
$\newcommand{\nhtnd}{n_{\rm H_{2}}}$
$\newcommand{\nhtcdpeak}{N_{\rm H_{2}}^{peak}}$
$\newcommand{\ncchcd}{N_{\rm CCH}}$
$\newcommand{\nnhtcd}{N_{\rm NH_{3}}}$
$\newcommand{\pnhtcd}{p_{N_{\rm H_{2}}}}$
$\newcommand{\pdustt}{p_{T_{\rm dust}}}$
$\newcommand{\vlsr}{\rm v_{lsr}}$
$\newcommand{\vinfall}{\rm v_{infall}}$
$\newcommand{\mjth}{M_{\rm J}^{\rm th}}$
$\newcommand{\lambdajth}{\lambda_{\rm J}^{\rm th}}$
$\newcommand{\mjthclump}{M_{\rm J, clp}^{\rm th}}$
$\newcommand{\lambdajthclump}{\lambda_{\rm J, clp}^{\rm th}}$
$\newcommand{\mjtotclump}{M_{\rm J, clp}^{\rm tot}}$
$\newcommand{\lambdajtotclump}{\lambda_{\rm J, clp}^{\rm tot}}$
$\newcommand{\mjflowclump}{M_{\rm J, clp}^{\rm com, flow}}$
$\newcommand{\lambdajflowclump}{\lambda_{\rm J, clp}^{\rm com, flow}}$
$\newcommand{\mjthens}{M_{\rm J, ens}^{\rm th}}$
$\newcommand{\lambdajthens}{\lambda_{\rm J, ens}^{\rm th}}$
$\newcommand{\Pex}{P_{\rm ex}}$
$\newcommand{\Pin}{P_{\rm in}}$
$\newcommand{\mach}{\mathcal{M}}$
$\newcommand{\mjtu}{M_{\rm J}^{\rm tu}}$
$\newcommand{\lambdajtu}{\lambda_{\rm J}^{\rm tu}}$
$\newcommand{\av}{A_{\rm v}}$
$\newcommand$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{$\tex$ttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{$\tex$trm{#1}}$
$\newcommand{$\hl$}[1]{$\tex$trm{#1}}$
$\newcommand{$\hi$}{H~$\textsc${i}}$
$\newcommand{$\hi$i}{H~$\textsc${ii}}$
$\newcommand{$\msun$}{\rm M_\odot}$
$\newcommand{$\lsun$}{\rm L_\odot}$
$\newcommand{$\kms$}{km~s^{-1}}$
$\newcommand{$\jybeam$}{Jy beam^{-1}}$
$\newcommand{$\mjybeam$}{mJy beam^{-1}}$
$\newcommand{$\hmole$}{H_2}$
$\newcommand{$\cm$}{cm^{-2}}$
$\newcommand{$\pressure$}{K~cm^{-3}}$
$\newcommand{$\hcop$}{HCO^+}$
$\newcommand{$\hcop$one}{HCO^+~J=1-0}$
$\newcommand{$\htcop$}{H^{13}CO^+}$
$\newcommand{$\htcop$one}{H^{13}CO^+~J=1-0}$
$\newcommand{$\hcn$}{HCN}$
$\newcommand{$\hcn$one}{HCN~J=1-0}$
$\newcommand{$\htcn$}{H^{13}CN}$
$\newcommand{$\htcn$one}{H^{13}CN~J=1-0}$
$\newcommand{$\cs$}{CS}$
$\newcommand{$\cs$two}{CS~J=2-1}$
$\newcommand{$\sio$}{SiO}$
$\newcommand{$\sio$two}{SiO~J=2-1}$
$\newcommand{$\sio$five}{SiO~J=5-4}$
$\newcommand{$\hctn$}{HC_3N}$
$\newcommand{$\hctn$ele}{HC_3N~J=11-10}$
$\newcommand{$\hfourtyalpha$}{H40\alpha}$
$\newcommand{$\chtoh$}{CH_3OH}$
$\newcommand{$\chtoh$line}{CH_3OH 2(1,1)-1(1,0)A}$
$\newcommand{$\cch$}{CCH}$
$\newcommand{$\cch$line}{CCH~N_{J,F}=1_{3/2,2}-0_{1/2,1}}$
$\newcommand{$\cch$hyperfine}{CCH~N_{J,F}=1_{3/2,1}-0_{1/2,0}}$
$\newcommand{$\so$}{SO}$
$\newcommand{$\so$line}{SO v =0, 3(2)-2(1)}$
$\newcommand{$\nht$}{NH_3}$
$\newcommand{$\mclump$}{M_{\rm clp}}$
$\newcommand{$\rclump$}{r_{\rm clp}}$
$\newcommand{$\RGC$}{R_{\rm GC}}$
$\newcommand{$\mvir$}{M_{\rm vir}}$
$\newcommand{$\tkin$}{T_{\rm kin}}$
$\newcommand{$\tex$}{T_{\rm ex}}$
$\newcommand{$\tbg$}{T_{\rm bg}}$
$\newcommand{$\tr$}{T_{\rm r}}$
$\newcommand{$\te$}{T_{\rm e}}$
$\newcommand{$\hcop$abundance}{\chi($\hcop$)}$
$\newcommand{$\hncabundance$}{\chi(\hnc)}$
$\newcommand{$\nthpabundance$}{\chi(\nthp)}$
$\newcommand{$\nthphcopabundance$}{\chi($\hcop$)/\chi(\nthp)}$
$\newcommand{$\nthphncabundance$}{\chi(\hnc)/\chi(\nthp)}$
$\newcommand{$\tcoab$}{\chi(^{13}CO)}$
$\newcommand{$\ceoab$}{\chi(C^{18}O)}$
$\newcommand{$\co$}{CO}$
$\newcommand{$\tco$}{^{13}CO}$
$\newcommand{$\ceo$}{C^{18}O}$
$\newcommand{$\co$one}{^{12}CO J=1-0}$
$\newcommand{$\co$two}{^{12}CO J=2-1}$
$\newcommand{$\co$thr}{^{12}CO J=3-2}$
$\newcommand{$\tco$one}{^{13}CO J=1-0}$
$\newcommand{$\tco$two}{^{13}CO J=2-1}$
$\newcommand{$\tco$thr}{^{13}CO J=3-2}$
$\newcommand{$\ceo$one}{C^{18}O~J=1-0}$
$\newcommand{$\ceo$two}{C^{18}O J=2-1}$
$\newcommand{$\ceo$thr}{C^{18}O J=3-2}$
$\newcommand{$\degree$}{^{\circ}}$
$\newcommand{$\nht$cd}{N_{\rm H_{2}}}$
$\newcommand{$\dustt$}{T_{\rm dust}}$
$\newcommand{$\nht$nd}{n_{\rm H_{2}}}$
$\newcommand{$\nht$cdpeak}{N_{\rm H_{2}}^{peak}}$
$\newcommand{$\ncchcd$}{N_{\rm CCH}}$
$\newcommand{$\nnhtcd$}{N_{\rm NH_{3}}}$
$\newcommand{$\pnhtcd$}{p_{N_{\rm H_{2}}}}$
$\newcommand{$\pdustt$}{p_{T_{\rm dust}}}$
$\newcommand{$\vlsr$}{\rm v_{lsr}}$
$\newcommand{$\vinfall$}{\rm v_{infall}}$
$\newcommand{$\mjth$}{M_{\rm J}^{\rm th}}$
$\newcommand{$\lambdajth$}{\lambda_{\rm J}^{\rm th}}$
$\newcommand{$\mjth$clump}{M_{\rm J, clp}^{\rm th}}$
$\newcommand{$\lambdajth$clump}{\lambda_{\rm J, clp}^{\rm th}}$
$\newcommand{$\mjtotclump$}{M_{\rm J, clp}^{\rm tot}}$
$\newcommand{$\lambdajtotclump$}{\lambda_{\rm J, clp}^{\rm tot}}$
$\newcommand{$\mjflowclump$}{M_{\rm J, clp}^{\rm com, flow}}$
$\newcommand{$\lambdajflowclump$}{\lambda_{\rm J, clp}^{\rm com, flow}}$
$\newcommand{$\mjth$ens}{M_{\rm J, ens}^{\rm th}}$
$\newcommand{$\lambdajth$ens}{\lambda_{\rm J, ens}^{\rm th}}$
$\newcommand{$\Pex$}{P_{\rm ex}}$
$\newcommand{$\Pin$}{P_{\rm in}}$
$\newcommand{$\mach$}{\mathcal{M}}$
$\newcommand{$\mjtu$}{M_{\rm J}^{\rm tu}}$
$\newcommand{$\lambdajtu$}{\lambda_{\rm J}^{\rm tu}}$
$\newcommand{$\av$}{A_{\rm v}}$
$\newcommand$
$\newcommand{$\arraystretch$}{1.0}$
$\newcommand{$\arraystretch$}{1.0}$
$\newcommand{$\arraystretch$}{1.0}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# ) clump

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.01937-b31b1b.svg)](https://arxiv.org/abs/2301.01937)<mark>Appeared on: 2023-01-06</mark> - _34 pages with 20 figures, Accepted by MNRAS on 2022 December 28_

</div>
<div id="authors">

<mark><mark>S. Zhang</mark></mark>, et al. -- incl., <mark><mark>T. Liu</mark></mark>, <mark><mark>H. Liu</mark></mark>, <mark><mark>S. Li</mark></mark>, <mark><mark>Q. Zhang</mark></mark>, <mark><mark>C. W. Lee</mark></mark>, <mark><mark>E. Vázquez-Semadeni</mark></mark>

</div>
<div id="abstract">

**Abstract:** Whether ionization feedback triggers the formation of massive stars is highly debated. Using ALMA 3 mm observations with a spatial resolution of$\sim0.05$pc and a mass sensitivity of 1.1$\msun$beam$^{-1}$at 20 K, we investigate the star formation and gas flow structures within the ionizing feedback-driven structure, a clump-scale massive ($\gtrsim1500$\msun) bright-rimmed cloud (BRC) associated with IRAS 18290$-$0924. This BRC is bound only if external compression from ionized gas is considered. A small-scale ($\lesssim1$pc) age sequence along the direction of ionizing radiation is revealed for the embedded cores and protostars, which suggests triggered star formation via radiation-driven implosion (RDI). Furthermore, filamentary gas structures converge towards the cores located in the BRC's center, indicating that these filaments are fueling mass towards cores. The local core-scale mass infall rate derived from$\htcop$oneblue profile is of the same order of magnitude as the filamentary mass inflow rate, approximately 1$\msun$kyr$^{-1}$. A photodissociation region (PDR) covering the irradiated clump surface is detected in several molecules, such as$\cch$,$\hcop$, and$\cs$whereas the spatial distribution stratification of these molecules is indistinct.$\cch$spectra of the PDR possibly indicate a photoevaporation flow leaving the clump surface with a projected velocity of$\sim2$\kms. Our new observations show that RDI accompanied by a clump-fed process is operating in this massive BRC. Whether this combined process works in other massive BRCs is worth exploring with dedicated surveys.

</div>

<div id="div_fig1">

<img src="tmp_2301.01937/./BRC-Statistics.png" alt="Fig1" width="100%"/>

**Figure 1. -** Summary of the published studies about RDI. The meanings of "SFO", "CG", "individual", "ALMAGAL $\hi$i", and "ALMAGAL BRC" can be found in Sect \ref{INTRODUCTION}. "BRC@CN20" \citep{Orgeta16} and "Pillars@CygnusX" \citep{Schneider16} highlight the two single-dish observations which have the clump mass of $> 1000$ $\msun$. The histograms show the number distributions of different types of studies. Dashed grids mark the mass of 10, $10^2$, and $10^3$ $\msun$ and the distances of 1 and 3 kpc. The mass and distance of ALMAGAL sources are taken from \citet{Urquhart18}. The mass for other sources is mainly derived from either continuum emission of dust or line emission of molecules, e.g. $\tco$ and $\ceo$. Therefore, the uncertainty of the mass estimation for all sources in the figure is difficult to unify, and thus should be cautioned. The distance for simulation work is assumed to be the averaged distance of the entire sample because many simulations do not set the distance. The detailed information for the whole sample in this figure is in Appendix \ref{App:Figure1Information}. (*StatisticsMap*)

</div>
<div id="div_fig2">

<img src="tmp_2301.01937/./PV-View.png" alt="Fig10" width="100%"/>

**Figure 10. -** $\cch$line emission. $\tex$tit{Panel a:}$\cch$ integrated intensity maps overlaid with the contours of 8 \micron emission (black) and the 3 mm continuum (magenta) with levels the same as $\tex$tit{Panel a} of Fig. \ref{StarFormationFigure}. The white line indicates the 8-\micron rim spine extracted by $\tex$ttt{Filfinder} in Sect. \ref{ENVIRONMENTSANDSTATUS}. The numbers on the spine indicate the offset positions corresponding to $\tex$tit{Panel b}. $\tex$tit{Panel b:} PV cut along the rim spine. The three black and gray lines show the $\tex$ttt{MCMC}-fitted compressed and PeF components with velocity centroid  and dispersion, respectively.  $\tex$tit{Panel c:} PV cut (width $=5$\arcsec) along the three arrows shown in $\tex$tit{Panel a}. $\tex$tit{Panel d:} Intensity profiles of various species along the arrows. In $\tex$tit{Panels c} and $\tex$tit{d}, the zero offset positions are positions at the spine and a positive offset means a position close to the interior of the clump. (*PDRDynamics*)

</div>
<div id="div_fig3">

<img src="tmp_2301.01937/./Large-Env-Improved.png" alt="Fig2" width="100%"/>

**Figure 2. -** Environment of I18290. $\tex$tit{Panel a:}$\tex$tit{Spitzer} 24/8.0/4.5 \micron RGB image overlaid with the contours of 20 cm (red dashed) and ATLASGAL 870 \micron(blue) continuum. The levels of 20 cm continuum are [4, 5, 6, 7, 8, 9, 10, 12, 14, 16] $\times \sigma_{\rm 20 cm}$, where noise of the 20 cm image $\sigma_{\rm 20 cm} = 0.45$ $\mjybeam$. The red bold contour of 20 cm emission (level: 2.38 $\mjybeam$ = 40\% of the continuum peak) highlights the region considered in the IBL calculations. The levels of 870 \micron emission are [0.2, 0.3, 0.4, 0.5, 0.7, 0.9, 1.3, 1.8] $\jybeam$. The white and black circles indicate the ATOMS-imaged field and the clump effective radius $r_{\rm clp}$ given by \citet{Urquhart18}, respectively. The blue bold contour of 870 \micron continuum with a level of 0.3 $\mjybeam$ indicates a region with a size roughly equivalent to the circle size of $r_{\rm clp}$.  $\tex$tit{The small panel} shows the 870 \micron intensity profile along the black line. $\tex$tit{Panel b:} The 8 \micron emission overlaid with the contours of the $\tex$tit{Herschel} dust temperature $\dustt$(red) and column density $\nht$cd(blue), with levels of [21.3, 21.6, 21.9, 22.2, 23.2, 24.2, 25.2] K and [2.4, 2.7, 3, 4, 5, 6, 7, 8, 9, 10] $\times 10^{22}$ $\cm$, respectively. The dotted line marks the rim spine extracted by $\tex$ttt{Filfinder} from 8 \micron image. (*GlobalEnviron*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

155  publications in the last 7 days.
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers